In [11]:
from pathlib import Path
from datetime import date
import os
import load_csv
data_frame = load_csv.load("data/clean_data.csv")
context = load_csv.getContext()

In [12]:
data_frame.show()

+--------------------+----------------+-----------------+----------+--------------------+-------------------+----------------+---+------+----------+--------------------+----------------+----------------------+----------------+-------------------+
|            order_id|     customer_id|    customer_name|product_id|        product_name|   product_category|    payment_type|qty| price|  datetime|             country|            city|ecommerce_website_name|  payment_txn_id|payment_txn_success|
+--------------------+----------------+-----------------+----------+--------------------+-------------------+----------------+---+------+----------+--------------------+----------------+----------------------+----------------+-------------------+
|2NEBnZETy1FdFpKJf...|wEwuo1GDdnbC8ju5|   STEPHEN THOMAS|     14315|ESR Boost Kicksta...|tv, audio & cameras|             UPI|  3| 25.19|2022-05-19|       United States|       Blackhawk|       www.wayfair.com|XXFqXfizoOtsQ0S1|                  Y|
|cs4mesS4QiK

In [13]:
data_frame.createOrReplaceTempView('orders')

In [19]:
# year_column = date_part(lit('YEAR'),'datetime').alias('year')
# month_column = date_part(lit('month'),'datetime').alias('month')
# day_column = date_part(lit('D'), 'datetime').alias('day')
# data_frame = data_frame.withColumn('qty', col('qty').cast('integer'))
# df_test = data_frame.select(data_frame.datetime, year_column, month_column, day_column, data_frame.country, data_frame.product_name, data_frame.qty)
# products = data_frame.select()
# df_test.groupBy('product_name').sum('qty').sort('country').show()
# df_test.groupBy('country').agg({'qty':'sum'}).show()#ps_sum(data_frame.qty)).show()
# df_test.groupBy(['year', 'month','product_name']).agg({'qty':'sum'}).show()
# df_test.agg(countDistinct('product_name')).show()
# df_test_2 = df_test.sort('datetime')
# df_test_2.show()
# df_test_2.agg()
query = 'SELECT product_id, product_name, SUM(qty), datetime FROM orders GROUP BY product_name'
context.sql(query).show()


AnalysisException: [MISSING_AGGREGATION] The non-aggregating expression "product_id" is based on columns which are not participating in the GROUP BY clause.
Add the columns or the expression to the GROUP BY, aggregate the expression, or use "any_value(product_id)" if you do not care which of the values within a group is returned.;
Aggregate [product_name#422], [product_id#421, product_name#422, sum(qty#425) AS sum(qty)#592L, datetime#427]
+- SubqueryAlias orders
   +- View (`orders`, [order_id#418,customer_id#419,customer_name#420,product_id#421,product_name#422,product_category#423,payment_type#424,qty#425,price#426,datetime#427,country#428,city#429,ecommerce_website_name#430,payment_txn_id#431,payment_txn_success#432])
      +- Relation [order_id#418,customer_id#419,customer_name#420,product_id#421,product_name#422,product_category#423,payment_type#424,qty#425,price#426,datetime#427,country#428,city#429,ecommerce_website_name#430,payment_txn_id#431,payment_txn_success#432] csv


In [61]:

print(os.path.abspath('') + "/test")

/home/miguel674-rev/2009_Big_Data/project-2/Revature_Project_2/data_analysis_and_cleaner/notebooks/test
